# Time-Resolved Carbon Factors

This notebook combines Energy Hub modelling in BESOS with analysis of the wider power system using SILVER [[1]].
SILVER solves the optimal power flow problem for an entire electricty grid.
From this we can extract the carbon emission every hour, which depends on the generation mix at that moment.
This is obtained by running `uc_results_analysis_Storage_Remuneration_w_mustrun.py` to obtain `hourly_GHG_emission.csv`.
After some processing, this gives the carbon factor time series that is used in place of the annual-averaged carbon factors typically used in the Energy Hub model.

[1]:https://doi.org/10.1016/j.energy.2017.07.027

In [ ]:
from pprint import pprint as pp

import pandas as pd
from pyehub.energy_hub.ehub_model import EHubModel

## Importing CSV
First the .csv file is read in and the date column is given a column name.
The type of each generator is removed from the table as the type is not important for PyEHub.

In [ ]:
raw_data = pd.read_csv("hourly_GHG_emission.csv")
raw_data.rename(columns={"Unnamed: 0": "Date"}, inplace=True)
raw_data = raw_data.set_index("Date")
raw_data = raw_data.drop(["kind"], axis=0)
raw_data.head()

## CSV Reformating
To ensure no type errors all the columns are converted into floats.
The index is switched to the time steps used by PyEHub and the specific dates are removed.
*This can be changed depending on how the particular model handles the timesteps*.
A unit conversion ensures the carbon factors are in the correct units for the PyEHub model.
The output is then converted into a dictionary where the key is each timestep and the value is the hourly carbon factor.

In [ ]:
data = raw_data.apply(pd.to_numeric)

data = data.reset_index()
data = data.drop(["Date"], axis=1)
# data=data[:24] # first day only
data = data.mean(axis=1) / 100000
dict_w_index = data.to_dict()

carbon_factor_input = dict_w_index

## Creating a PyEHub model
The PyEHub model is created normally.
To show the difference in results the unmodified example model is solved.

In [ ]:
hub = EHubModel(excel="test_file_CO2.xlsx")
oldhub = hub.solve()

## Modifying Carbon Factors and Time Series
The carbon factors are only applied to import streams and carbon credits are only applied to export streams.
The carbon factor for the particular stream is tied to a time series by overwritting the value of the carbon factor with the key for the time series.
The formated values from the csv are then inputted into the model as a time series.

In [ ]:
print("Import Streams:")
print(hub.import_streams)
print()

print("Carbon Factors Before:")
print(hub.CARBON_FACTORS)
print()
hub.CARBON_FACTORS["Grid"] = "Grid_co2_factor"
print("Carbon Factors After:")
print(hub.CARBON_FACTORS)
print()

print("Time Series Before:")
print(hub.TIME_SERIES)
print()
hub.TIME_SERIES["Grid_co2_factor"] = carbon_factor_input
print("Time Series After:")
print(hub.TIME_SERIES)

## Results
The model is then recompiled with the new time resolved carbon factors.
We can see that the outputs have changed.

In [ ]:
hub.recompile()
newhub = hub.solve()
print(oldhub["solution"]["total_carbon"])
print(newhub["solution"]["total_carbon"])

In [ ]:
pp(newhub["solution"])